In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io as scio
import time 

In [2]:
path_noisy = r'Input_Patches_3Dreal1.csv'

In [3]:
data = pd.read_csv(path_noisy, header=None)

In [4]:
data = data.T

In [6]:
data = data.astype(np.float32)
data= torch.from_numpy(data.values)

In [7]:
w1 = 12
w2 = 12
w3 = 12

In [ ]:
train_size = int(len(data) * 0.8)
train = data[:train_size]
vaild = data[train_size:]
print(train.shape)
print(vaild.shape)
batch_size1 = 64
w1 = 12
w2 = 12
w3 = 12
train_data= TensorDataset(train)
vaild_data= TensorDataset(vaild)

train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle = True)

vaild_loader = torch.utils.data.DataLoader(vaild_data,
                                          batch_size = 64,
                                          shuffle = True)

torch.Size([10067, 1728])
torch.Size([2517, 1728])


In [ ]:
#Fully connected (FC) block
class FCB(nn.Module):
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fc = nn.Linear(input_size, output_size)
        self.activation = nn.LeakyReLU()
        self.bn = nn.BatchNorm1d(output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc(x)
        x = self.activation(x) 
        x = self.bn(x)
        x = self.dropout(x)
        
        return x

In [10]:
class PAB(nn.Module):  ##position attention block
    def __init__(self, input_size, output_size, dropout=0.1):
        super().__init__()
        
        self.fcb1 = FCB(input_size, output_size, dropout)
        self.fcb2 = FCB(input_size, output_size, dropout)
        self.fcb3 = FCB(input_size, output_size, dropout)
        self.fcb4 = FCB(input_size, output_size, dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x1 = self.fcb1(x)
        x2 = self.fcb2(x)
        x3 = self.fcb3(x)
        x4 = self.fcb4(x)
        

        x = x1*x2
        x = self.softmax(x)
        x = x*x3
        x = x+x4
        
        return x   

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_size, dropout=0.1):
        super().__init__()

        self.cov1d = nn.Conv1d(in_channels=1, out_channels= 64 ,kernel_size = 5, padding=0, stride=1)

        self.pab1 = PAB(int(input_size-5+1), 128, dropout)
        self.fcb1 = FCB(128, 128, dropout)

        self.pab2 = PAB(128, 64, dropout)
        self.fcb2 = FCB(64, 64, dropout)

        self.pab3 = PAB(64, 32, dropout)
        self.fcb3 = FCB(32, 32, dropout)

        self.pab4 = PAB(32, 16, dropout)
        self.fcb4 = FCB(16, 16, dropout)

        self.pab5 = PAB(16, 8, dropout)
        self.fcb5 = FCB(8, 8, dropout)

        self.pab6 = PAB(8, 4, dropout)
        self.fcb6 = FCB(4, 4, dropout)
        
        self.fcb7 = FCB(4, 4, dropout)
        self.pab7 = PAB(4, 4, dropout)
        

        


    def forward(self, x):

        x = x.reshape(x.shape[0],1,x.shape[1])
        x = self.cov1d(x)
        x = x.permute(0,2,1)
        x = nn.AdaptiveAvgPool1d(1)(x)
        x = x.squeeze()

        x1 = self.pab1(x)
        x13 = self.fcb1(x1)

        x2 = self.pab2(x1)
        x12 = self.fcb2(x2)

        x3 = self.pab3(x2)
        x11 = self.fcb3(x3)

        x4 = self.pab4(x3)
        x10 = self.fcb4(x4)

        x5 = self.pab5(x4)
        x9 = self.fcb5(x5)

        x6 = self.pab6(x5)
        x8 = self.fcb6(x6)

        x7 = self.fcb7(x6)
        x7 = self.pab7(x7)
        

        

        return x7,x8,x9,x10,x11,x12,x13

In [ ]:
class CustomActivation(torch.autograd.Function):    
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        y = 7/3 * (x - torch.tanh(x)) * torch.cos(x/2)
        return y
    
    @staticmethod
    def backward(ctx, grad_output):
        x, = ctx.saved_tensors
        grad_x = grad_output * (7/3 * ((torch.cos(x/2)*(1 - torch.pow(torch.cosh(x), -2)))+ (x - torch.tanh(x))*(-0.5*torch.sin(x/2))))
                                       
        return grad_x
    
class CustomActivationModule(torch.nn.Module):
    def forward(self, x):
        return CustomActivation.apply(x)  
    

activation = CustomActivationModule()    

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, dropout=0.1):
        super().__init__()

        self.pab1 = PAB(8, 8, dropout)
        self.pab2 = PAB(16, 16, dropout)
        self.pab3 = PAB(32, 32, dropout)
        self.pab4 = PAB(64, 64, dropout)
        self.pab5 = PAB(128, 128, dropout)

        self.fc = nn.Linear(256,output_size)
        self.activation = activation


    def forward(self, x7,x8,x9,x10,x11,x12,x13):
        
        x = torch.cat((x7,x8),dim=1)
        x = self.pab1(x)
        x = torch.cat((x,x9),dim=1)
        x = self.pab2(x)
        x = torch.cat((x,x10),dim=1)
        x = self.pab3(x)
        x = torch.cat((x,x11),dim=1)
        x = self.pab4(x)
        x = torch.cat((x,x12),dim=1)
        x = self.pab5(x)
        x = torch.cat((x,x13),dim=1)
        x = self.fc(x)
        x = self.activation(x)
        return x

In [14]:
class AutoEncoder(nn.Module):
    def __init__(self,input_size, output_size):
        super().__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(output_size)

    def forward(self, x):
        x7,x8,x9,x10,x11,x12,x13= self.encoder(x)
        x = self.decoder(x7,x8,x9,x10,x11,x12,x13)

        return x

In [15]:
class WelschLoss(nn.Module):
    def __init__(self, delta):
        super().__init__()
        self.delta = delta

    def forward(self, y_pred, y_true):
        x = y_true - y_pred
        loss = 1 - torch.exp(-0.5 * torch.pow((x / self.delta),2))
        return torch.mean(loss)

In [16]:
class Loss(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        temp = output*N
        temp1 = data.shape[1]*(temp.sum(1))
        temp2 = (output.sum(1))*(N.sum(1))
        temp3 = torch.pow(data.shape[1]*((output**2).sum(1)) - ((output.sum(1))**2),1/2)
        temp4 = torch.pow(data.shape[1]*((N**2).sum(1))-((N.sum(1))**2),1/2) 
        loss = (temp1-temp2)/(temp3*temp4)
        a = torch.min(loss)
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))        
        return self.r*a+(1-self.r)*b

In [17]:
class Loss0(nn.Module):
    def __init__(self,delta,r):
        super().__init__()
        self.delta = delta
        self.r = r
    def forward(self,input,output):
        N = input-output
        b = torch.mean(1 - torch.exp(-0.5 * torch.pow((N / self.delta),2)))
        K = output.shape[0]
        y2_mean = torch.mean(output)
        n2_mean = torch.mean(N)
        covariance = 0.0
        variance_y2 = 0.0
        variance_n2 = 0.0
        for j in range(K):
            y2_sample = output[j, :]
            n2_sample = N[j, :]
            covariance += ((y2_sample - y2_mean) * (n2_sample - n2_mean))
            variance_y2 += ((y2_sample - y2_mean) ** 2)
            variance_n2 += ((n2_sample - n2_mean) ** 2)
            lcc_value = (covariance / torch.sqrt(variance_y2 * variance_n2)) ** 2
            lcc_value = torch.mean(lcc_value)
        return self.r*lcc_value+(1-self.r)*b

In [ ]:
device = torch.device("cuda") 
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
criterion = Loss0(delta=0.55,r=0.1)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
loss_0 = []
start_time = time.time()
es_cnt = 0
es_thres = 5
prev_train_loss = float('inf')
loss_train = []
loss_vaild = []
num_epochs = 100 
for epoch in range(num_epochs):
  train_loss = 0.0
  
  for i , (batch) in enumerate(train_loader):

    train_batch = batch[0].to(device)
    
    optimizer.zero_grad()
    outputs = model(train_batch)
    loss = criterion(outputs, train_batch)
    loss.backward() 
    optimizer.step()

    loss_0.append(loss.item())

    train_loss += loss.item()
  train_loss = train_loss/(np.ceil(train.size(0)/batch_size1))
  loss_train.append(train_loss)
    



  valid_loss = 0.0
  with torch.no_grad():
    for i , (batch) in enumerate(vaild_loader):
    
      val_batch = batch[0].to(device)
      
      outputs = model(val_batch)
      loss = criterion(outputs, val_batch)
      valid_loss += loss.item()
    valid_loss = valid_loss/(np.ceil(vaild.size(0)/batch_size1))
    loss_vaild.append(valid_loss)
    print("Epoch [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}".format(epoch+1, num_epochs, train_loss, valid_loss))

    
    # Early stopping
    if train_loss - prev_train_loss >= 0:
        es_cnt += 1
    else:
        pass

    if es_cnt >= es_thres:
        print(f"Early stopped at epoch {epoch}, train loss stop improving")
        break  
    




    prev_train_loss = train_loss
  print('loss_train: ', loss_train)
  print('vaild_train: ',loss_vaild)          
print("Training finished")
current_time = time.time()
time_sum = current_time-start_time
print(time_sum)

Epoch [1/100], Train Loss: 0.3229, Valid Loss: 0.2671
loss_train:  [0.3229213319247282]
vaild_train:  [0.2671116329729557]
Epoch [2/100], Train Loss: 0.2489, Valid Loss: 0.2352
loss_train:  [0.3229213319247282, 0.24886217128626908]
vaild_train:  [0.2671116329729557, 0.23518313094973564]
Epoch [3/100], Train Loss: 0.2285, Valid Loss: 0.2224
loss_train:  [0.3229213319247282, 0.24886217128626908, 0.2285256548018395]
vaild_train:  [0.2671116329729557, 0.23518313094973564, 0.22238983139395713]
Epoch [4/100], Train Loss: 0.2181, Valid Loss: 0.2138
loss_train:  [0.3229213319247282, 0.24886217128626908, 0.2285256548018395, 0.21812612733131723]
vaild_train:  [0.2671116329729557, 0.23518313094973564, 0.22238983139395713, 0.2137930255383253]
Epoch [5/100], Train Loss: 0.2111, Valid Loss: 0.2074
loss_train:  [0.3229213319247282, 0.24886217128626908, 0.2285256548018395, 0.21812612733131723, 0.21105555218609073]
vaild_train:  [0.2671116329729557, 0.23518313094973564, 0.22238983139395713, 0.213793025

In [20]:
loss_train = pd.DataFrame(loss_train)
loss_vaild = pd.DataFrame(loss_vaild)
loss = pd.concat([loss_train,loss_vaild],axis=1)

In [21]:
loss.to_csv(r'loss_3dreal1.csv',index=False)
torch.save(model.state_dict(), r'.\model_3dreal1.pth')

In [22]:
model = AutoEncoder(w1*w2*w3,w1*w2*w3).to(device)
data = data.to(device)
model.load_state_dict(torch.load(r'.\model_3dreal1.pth'))
model.eval()
with torch.no_grad():
    output = model(data)
    loss = criterion(output, data)
print(output.shape)

output = output.cpu()
output = output.numpy()
output = pd.DataFrame(output)
output.to_csv(r'./output_3dreal1.csv',index=None,header=None)

torch.Size([12584, 1728])
